# Microtraining - Data-Analysis with Keras and Tensorflow

In [ ]:
## Exercise 1

In [1]:
#imports
import tensorflow as tf

In [2]:
# Load dataset
mnist = tf.keras.datasets.mnist

In [5]:
# Input Pipeline


(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


In [6]:
# Inference

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [7]:
# Model compilation
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
# Training

model.fit(x_train, y_train, epochs=5)


Epoch 1/5
60000/60000 [==============================] - 20s 340us/step - loss: 0.2014 - acc: 0.9395
Epoch 2/5
60000/60000 [==============================] - 37s 618us/step - loss: 0.0819 - acc: 0.9749
Epoch 3/5
60000/60000 [==============================] - 37s 622us/step - loss: 0.0518 - acc: 0.9839
Epoch 4/5
60000/60000 [==============================] - 38s 633us/step - loss: 0.0367 - acc: 0.9883
Epoch 5/5
60000/60000 [==============================] - 36s 606us/step - loss: 0.0281 - acc: 0.9905


In [9]:
# Test

model.evaluate(x_test, y_test)

10000/10000 [==============================] - 1s 94us/step


[0.0693634459810739, 0.9797]

Todo:
* add more layers to your network and compare the accuracy before and after
* run your training phase with 10 instead of 3 epochs. Explain why the accuracy is dropping. Try adding a dropout layer afterwards
* change your train test ratio